In [3]:
import os
import scanpy as sc
from scipy.io import mmread
from scipy.sparse import csr_matrix

inputfile = "../data/AllonKleinLab/Experiment2/stateFate_inVivo_normed_counts.mtx"
outputfolder = "../out/AllonKleinLab/Experiment2"

HVF = True
TEST = False

TEST_genes = 300
TEST_samples = 500

k = 15

 # Load Dataset

In [4]:
se = csr_matrix(mmread(inputfile))
se = se[se.getnnz(1) > 0][:, se.getnnz(0) > 0]

if TEST:
    tsamples = min(TEST_samples, se.shape[0])
    tgenes = min(TEST_genes, se.shape[1])
    se.resize((tsamples, tgenes))
    se = se[se.getnnz(1) > 0][:, se.getnnz(0) > 0]

adata = sc.AnnData(se)
sc.pp.normalize_total(adata, target_sum=1e6) #TODO check this
sc.pp.log1p(adata)
sc.pp.scale(adata, zero_center=False)
sc.pp.highly_variable_genes(adata, flavor="seurat")
if HVF:
    adata = adata[:, adata.var.highly_variable]
    adata = adata[adata.X.getnnz(1) > 0][:, adata.X.getnnz(0) > 0]
    print(adata)

sc.pp.pca(adata)
sc.pp.neighbors(adata)
sc.tl.umap(adata)

: 

 # Visualize data

In [ ]:
# PCA plot
imgname = os.path.join(outputfolder, "PCA.png")
print(f"Saving Image --- {imgname}")
sc.pl.pca(adata, save=imgname.split("/")[-1])

In [ ]:
# UMAP plot
imgname = os.path.join(outputfolder, "UMAP.png")
print(f"Saving Image --- {imgname}")
sc.pl.umap(adata, save=imgname.split("/")[-1])

In [ ]:
# Elbow plot
imgname = os.path.join(outputfolder, "elbow.pdf")
print(f"Saving Image --- {imgname}")
sc.pl.pca_variance_ratio(adata, log=True, save=imgname.split("/")[-1])

In [ ]:
## Archetypes
# import archetypes as arch
# from time import time
#
# aa_kwargs = {
#    "n_archetypes": 4,
#    "n_init": 5,
#    "max_iter": 10000,
#    "verbose": True,
#    "tol": 1e-3,
# }
#
# mod0 = arch.AA(**aa_kwargs, algorithm_init="furthest_sum")
#
# t0=time()
# mod0.fit_transform(adata.X.toarray())
# t1=time()
#
# print(f"mod0: {t1-t0:.2f} seconds|RSS: {mod0.rss:.2f}")